In [1]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
import numpy as np
import requests
from talib import MACD, RSI, BBANDS
from datetime import datetime, timedelta, date
from IPython.display import display, HTML
%matplotlib inline
import json
import math
from dateutil import parser
from pytz import timezone
import pytz
import sys
import iso8601
from pandas import DataFrame as df

In [2]:
batch_id = "075cb735-f187-4e19-ba65-b6beadedd080"

In [3]:
db_conn = create_engine("postgresql://momentum@localhost:5432/tradedb")

In [4]:
api = tradeapi.REST(base_url="https://api.alpaca.markets")
session = requests.session()

In [5]:
query = f'''
SELECT t.* 
FROM new_trades as t, algo_run as r
WHERE 
    t.algo_run_id = r.algo_run_id and
    r.batch_id = '{batch_id}' AND
    t.expire_tstamp is null 
ORDER BY symbol, tstamp
'''
trades =  pd.read_sql_query(query, db_conn)
trades['client_time']= pd.to_datetime(trades['client_time'])

In [6]:
query = f'''
SELECT * 
FROM 
algo_run as t
WHERE 
    batch_id = '{batch_id}'
    
ORDER BY algo_run_id
'''
algo_runs = pd.read_sql_query(query, db_conn)
algo_runs.set_index('algo_run_id', inplace=True)

In [7]:
algo_runs

,algo_name,algo_env,build_number,parameters,start_time,end_time,end_reason,batch_id,ref_algo_run
algo_run_id,,,,,,,,,
4769,momentum_long,BACKTEST,0.0.55,{'DSN': 'postgresql://momentum@localhost/trade...,2020-10-06 01:56:54.619449,None,None,075cb735-f187-4e19-ba65-b6beadedd080,4740
4770,vwap_short,BACKTEST,0.0.55,{'DSN': 'postgresql://momentum@localhost/trade...,2020-10-06 01:56:54.924628,None,None,075cb735-f187-4e19-ba65-b6beadedd080,4740


In [8]:
query = f'''
SELECT a2.start_time 
FROM algo_run as a1, algo_run as a2
WHERE 
    a1.batch_id = '{batch_id}' AND
    a1.ref_algo_run = a2.algo_run_id
ORDER BY start_time
'''
start_times  =  pd.read_sql_query(query, db_conn)

In [9]:
start_times

,start_time
0,2020-10-02 14:21:49.765661
1,2020-10-02 14:21:49.765661


In [10]:
start_day_to_analyze = start_times['start_time'][0].replace(hour=0, minute=0)

In [11]:
trades

,trade_id,algo_run_id,symbol,operation,qty,price,indicators,client_time,tstamp,stop_price,target_price,expire_tstamp
0,40115,4769,BBBY,buy,115,20.26,"{'avg': 19.70456742755804, 'macd': [0.05172919...",2020-10-02 10:28:00-04:00,2020-10-06 01:57:00.635806,19.39,22.86,None
1,40116,4769,BBBY,sell,115,20.72,"{'avg': 20.110579956204624, 'rsi': [59.7833889...",2020-10-02 11:24:00-04:00,2020-10-06 01:57:01.055037,19.39,22.86,None
2,40117,4769,GALT,buy,966,2.75,"{'avg': 2.6740382524570023, 'macd': [0.0115712...",2020-10-02 10:22:00-04:00,2020-10-06 01:57:36.808029,2.65,3.06,None
3,40118,4769,GALT,sell,966,2.64,"{'avg': 2.6837698234514153, 'rsi': [39.8642938...",2020-10-02 13:55:00-04:00,2020-10-06 01:57:37.172063,2.65,3.06,None
4,40119,4769,PRPO,buy,452,2.55,"{'avg': 2.457808019458626, 'macd': [0.01797201...",2020-10-02 10:27:00-04:00,2020-10-06 01:57:47.471415,2.33,3.21,None
5,40120,4769,PRPO,sell,226,2.67,"{'avg': 2.567608382153938, 'rsi': [69.97896304...",2020-10-02 10:35:00-04:00,2020-10-06 01:57:47.658135,2.33,3.21,None
6,40121,4769,PRPO,sell,226,2.55,"{'avg': 2.578615703344527, 'rsi': [54.88371506...",2020-10-02 10:39:00-04:00,2020-10-06 01:57:47.855018,2.33,3.21,None


In [12]:
trade_details = {}
batch = {}
for index, row in trades.iterrows():
    algo_run_id = row["algo_run_id"]
    symbol = row["symbol"]
    time = row["tstamp"]
    operation = row["operation"]
    price = row["price"]
    indicators = row["indicators"]
    qty = row["qty"]
    stop_price = row["stop_price"]
    target_price = row["target_price"]
    if algo_run_id not in trade_details:
        trade_details[algo_run_id] = {}
    if symbol not in trade_details[algo_run_id]:
        trade_details[algo_run_id][symbol] = []
    trade_details[algo_run_id][symbol].append(
        (operation, time, price, qty, indicators, stop_price, target_price)
    )
    if batch_id not in batch:
        batch[batch_id] = []
    if algo_run_id not in batch[batch_id]:
        batch[batch_id].append(algo_run_id)

In [13]:
symbol_name = []
counts = []
revenues = []

how_was_my_day = pd.DataFrame()
how_was_my_day["symbol"] = trades.symbol.unique()
how_was_my_day["revenues"] = how_was_my_day["symbol"].apply(calc_revenue)
how_was_my_day["count"] = how_was_my_day["symbol"].apply(count_trades)

NameError: name 'calc_revenue' is not defined

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
how_was_my_day

In [ ]:
round(how_was_my_day['revenues'].sum(),2)

In [ ]:
minute_history = {}
for batch_id, count in batch.items():
    for run_id in batch[batch_id]:
        symbols = trades.loc[trades["algo_run_id"] == run_id]["symbol"].value_counts()
        for symbol, count in symbols.items():
            if symbol not in minute_history:
                minute_history[symbol] = api.polygon.historic_agg_v2(
                    symbol,
                    1,
                    "minute",
                    _from=str((start_day_to_analyze - timedelta(days=8)).date()),
                    to=str((start_day_to_analyze + timedelta(days=1)).date()),
                ).df
                print(
                    f"{symbol} loaded {len(minute_history[symbol].index)} agg data points"
                )

In [ ]:
def grouper(iterable):
    prev = None
    group = []
    for item in iterable:

        if not prev or -0.02 <= float(item - prev) / prev <= 0.02:
            group.append(item)
        else:
            yield group
            group = [item]
        prev = item
    if group:
        yield group


def find_resistance(current_value: float, minute_history, now):
    """calculate next resistance"""
    est = pytz.timezone("US/Eastern")
    now = pd.Timestamp(now.astimezone(est))
    minute_history_index = minute_history["close"].index.get_loc(now, method="nearest")
    back_time = pd.Timestamp(now.to_pydatetime() - timedelta(days=4))
    back_time_index = minute_history["close"].index.get_loc(back_time, method="nearest")
    for back_track_min in range(minute_history_index + 1 - 120, back_time_index, -5):
        series = (
            minute_history["close"][back_track_min : minute_history_index + 1]
            .dropna()
            .between_time("9:30", "16:00")
            .resample("5min")
            .max()
        ).dropna()

        diff = np.diff(series.values)
        high_index = np.where((diff[:-1] >= 0) & (diff[1:] <= 0))[0] + 1
        if len(high_index) > 0:
            local_maximas = sorted(
                [series[i] for i in high_index if series[i] >= (current_value)]
            )
            if len(local_maximas) > 0:

                return local_maximas

    return None


def find_support(current_value, minute_history, now):
    """calculate support"""
    est = pytz.timezone("US/Eastern")
    now = pd.Timestamp(now.astimezone(est))
    minute_history_index = minute_history["close"].index.get_loc(now, method="nearest")
    back_time = pd.Timestamp(now.to_pydatetime() - timedelta(days=4))
    back_time_index = minute_history["close"].index.get_loc(back_time, method="nearest")
    for back_track_min in range(minute_history_index + 1 - 120, back_time_index, -5):
        series = (
            minute_history["close"][back_track_min : minute_history_index + 1]
            .dropna()
            .between_time("9:30", "16:00")
            .resample("5min")
            .max()
        ).dropna()
        diff = np.diff(series.values)
        high_index = np.where((diff[:-1] <= 0) & (diff[1:] > 0))[0] + 1
        if len(high_index) > 0:
            local_maximas = sorted(
                [series[i] for i in high_index if series[i] <= current_value]
            )
            if len(local_maximas) > 0:

                return local_maximas

    return None

In [ ]:
for symbol in minute_history:
    symbol_df = trades.loc[trades["symbol"] == symbol]
    start_date = symbol_df["client_time"].min()
    start_date = start_date.replace(hour=9, minute=30)
    end_date = start_date.replace(hour=16, minute=0)
    print(start_date)
    print()
    cool_down_date = start_date + timedelta(minutes=5)
    minute_history_index = minute_history[symbol]["close"].index.get_loc(
        start_date, method="nearest"
    )
    end_index = minute_history[symbol]["close"].index.get_loc(
        end_date, method="nearest"
    )
    cool_minute_history_index = minute_history[symbol]["close"].index.get_loc(
        cool_down_date, method="nearest"
    )
    open_price = minute_history[symbol]["close"][cool_minute_history_index]
    plt.plot(
        minute_history[symbol]["close"][minute_history_index:end_index].between_time(
            "9:30", "16:00"
        ),
        label=symbol,
    )
    plt.xticks(rotation=45)

    delta = 0
    profit = 0

    operations = []
    deltas = []
    profits = []
    times = []
    prices = []
    qtys = []
    indicators = []
    target_price = []
    stop_price = []
    resistances = []
    supports = []
    daily_change = []
    precent_vwap = []
    patterns = []
    for index, row in symbol_df.iterrows():
        if (
            row["operation"] == "buy"
            and row["qty"] > 0
            or row["operation"] == "sell"
            and row["qty"] < 0
        ):
            try:
                now = int(row["client_time"])
                continue
            except Exception:
                pass

            resistance = find_resistance(
                row["price"], minute_history[symbol], row["client_time"]
            )
            if resistance:
                for r in resistance:
                    plt.axhline(y=r, color="r")
            support = find_support(
                row["price"], minute_history[symbol], row["client_time"]
            )
            if support:
                for s in support:
                    plt.axhline(y=s, color="g")

        delta = (
            row["price"]
            * row["qty"]
            * (1 if row["operation"] == "sell" and row["qty"] > 0 else -1)
        )
        profit += delta
        plt.scatter(
            row["client_time"].to_pydatetime(),
            row["price"],
            c="g" if row["operation"] == "buy" else "r",
            s=100,
        )
        deltas.append(round(delta, 2))
        profits.append(round(profit, 2))
        operations.append(row["operation"])
        times.append(pd.to_datetime(row["client_time"]))
        prices.append(row["price"])
        qtys.append(row["qty"])
        patterns.append(
            row["indicators"]["patterns"] if "patterns" in row["indicators"] else None
        )
        row["indicators"].pop("patterns", None)
        indicators.append(row["indicators"])
        target_price.append(row["target_price"])
        stop_price.append(row["stop_price"])

        if resistance:
            resistances.append(resistance)
        else:
            resistances.append("")
        supports.append(support)
        daily_change.append(
            f"{round(100.0 * (row['price'] - open_price) / open_price, 2)}%"
        )
        precent_vwap.append(
            f"{round(100.0 * (row['indicators']['avg'] - open_price) / open_price, 2)}%"
            if "avg" in row["indicators"]
            else ""
        )

    d = {
        "profit": profits,
        "trade": deltas,
        "operation": operations,
        "at": times,
        "price": prices,
        "support": supports,
        "resistance": resistances,
        "qty": qtys,
        "daily change": daily_change,
        "vwap": precent_vwap,
        "indicators": indicators,
        "patterns": patterns,
        "target price": target_price,
        "stop price": stop_price,
    }
    print(f"{symbol} analysis with profit {round(profit, 2)}")
    display(HTML(pd.DataFrame(data=d).to_html()))
    plt.legend()
    plt.show()